In [1]:
%cd ../

/home/zulqarnain/develop/htfatorch


In [2]:
import logging
import numpy as np
import htfa_torch.dtfa as DTFA
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import htfa_torch.htfa as HTFA


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/utils/deprecation.py

In [3]:
from scipy.spatial import distance
from scipy import stats
import nilearn.plotting as niplot
import statsmodels.stats.multitest as smt
from ordered_set import OrderedSet
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score,StratifiedKFold
from sklearn.feature_selection import SelectPercentile, f_classif,SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.metrics import plot_roc_curve

In [4]:
import matplotlib
import matplotlib.pyplot as pl
from matplotlib import cycler
from seaborn.apionly import color_palette
column_width = 3.3
page_width = 6.7
# colorblind-friendly colors 
# source: https://personal.sron.nl/~pault/
colors = {
    'bright': ['4477AA', 
               '66CCEE', 
               '228833', 
               'CCBB44', 
               'EE6677', 
               'AA3377', 
               'BBBBBB'],
    'high_contrast': ['FFFFFF', 
                      'DDAA33', 
                      'BB5566', 
                      '004488', 
                      '00000'],
    'vibrant': ['0077BB', 
                '33BBEE', 
                '0099BB',
                'EE7733',
                'CC3311',
                'EE3377',
                'BBBBBB'],
    'muted': ['332288',
              '88CCEE',
              '44AA99',
              '117733',
              '999933',
              'DDCC77',
              'CC6677',
              '882255',
              'AA4499']
}
pl.rc('legend', frameon=False)
pl.rc('figure', figsize=(column_width, 
                         0.66 * column_width),
                dpi=120)
pl.rc('savefig', dpi=300)
pl.rc('font', size=8)


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [5]:
%matplotlib inline

In [6]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [7]:
affvids_db = niidb.FMriActivationsDb('data/affvids2018_mc_zrest.db')

In [8]:
dtfa = DTFA.DeepTFA([b for b in affvids_db.all() if 'rest' not in b.task], mask='/data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz', num_factors=100, embedding_dim=2)

02/03/2020 16:20:59 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub10_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 16:21:21 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub10_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 16:26:20 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub15_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 16:26:41 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub15_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 16:31:40 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub23_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 16:32:02 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub25_run01_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

02/03/2020 16:36:58 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub05_run01_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
02/03/2020 16:37:19 Loading Nifti image /data/zulqarnain/AffVids_mcwm/motion_corrected/sub05_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed i

In [9]:
dtfa.load_state('data/affvids_mc_norest_rz_k100_n1000_01232020_000625') 

In [10]:
validation_filter = affvids_db.inference_filter(training=False)
validation_blocks = [b for (b, block) in enumerate(dtfa._blocks) if validation_filter(block)]

In [11]:
def task_labeler(task):
    if 'heights' in task:
        return 'Heights'
    elif 'social' in task:
        return 'Social threat'
    elif 'spider' in task:
        return 'Spiders'
    elif 'rest' in task:
        return 'Rest'
    else:
        return None

def rest_task_labeler(task):
    if task == 'rest':
        return 'Rest'
    else:
        return 'Task'

def high_low_labeler(task):
    if 'high' in task:
        return 'high'
    elif 'low' in task:
        return 'low'
    else:
        return None

In [12]:
def block_task_labeler(block):
    return task_labeler(block.task)

def block_high_low_labeler(block):
    if 'high' in block.task:
        return 'high'
    elif 'low' in block.task:
        return 'low'
    else:
        return None

In [13]:
def subject_labeler(subject):
    return 'Subject %d' % subject

In [14]:
def block_subject_labeler(block):
    return subject_labeler(block.subject)

In [15]:
def classification_pipeline(data,targets,session_labels,cv=LeaveOneGroupOut(),
                                 scoring="roc_auc",feature_select=False,
                            feature_selection=SelectPercentile(f_classif, percentile=5),print_all=True):
    categories = np.unique(targets)
    overall_result = []
    overall_result_chance = []
    category_result = {key:None for key in categories}
    for category in categories:
        print("Processing %s" % (category))
        dummy_classifier = DummyClassifier()
        classification_target = [t==category for t in targets]
        svc = SVC(C=1., kernel="linear",probability=True,random_state=42)
        if feature_select:
            classification_target = np.array(classification_target)
            result = []
            result_chance = []
            for train_ind, test_ind in cv.split(data,classification_target,groups=session_labels):
                print("TRAIN:", train_ind, "TEST:", test_ind)
                train_ind = np.array(train_ind).astype(int)
                test_ind = np.array(test_ind).astype(int)
                training_labels = classification_target[train_ind]
                testing_labels = classification_target[test_ind]
                train_feature_classif = feature_selection.fit(data[train_ind],training_labels)
                training_data = train_feature_classif.transform(data[train_ind])
                testing_data = train_feature_classif.transform(data[test_ind])

                train_svc = svc.fit(training_data,training_labels)
                train_dummy = dummy_classifier.fit(training_data,training_labels)
                viz = plot_roc_curve(train_svc,testing_data,testing_labels)
                result.append(viz.roc_auc)
                pl.close()
                viz = plot_roc_curve(train_dummy,testing_data,testing_labels)
                result_chance.append(viz.roc_auc)
                pl.close()
        else:
            classifier = svc
            result = cross_val_score(
                classifier,
                data,
                classification_target,
                cv=cv,
                groups=session_labels,
                scoring="roc_auc",
            )

            result_chance = cross_val_score(
                dummy_classifier,
                data,
                classification_target,
                cv=cv,
                groups=session_labels,
                scoring="roc_auc",
            )
        result = np.array(result)
        result_chance = np.array(result_chance)
        category_result[category] = result
        overall_result.extend(result)
        overall_result_chance.extend(result_chance)
        if print_all:
            print("Scores: %1.4f +- %1.4f" % (
                result.mean(),
                result.std()))

            print("Chance Scores: %1.4f +- %1.4f" % (
                result_chance.mean(),
                result_chance.std()))
    
    if print_all:
        print ('...')
        print("Overall Score: %1.4f +- %1.4f" % (
        np.array(overall_result).mean(),
        np.array(overall_result).std()))

        print("Overall Dummy Score: %1.4f +- %1.4f" % (
        np.array(overall_result_chance).mean(),
        np.array(overall_result_chance).std()))

        print ("\n")
    return overall_result,overall_result_chance,category_result

In [16]:
dtfa_hyperparams = dtfa.variational.hyperparams.state_vardict()
all_weights_dtfa = dtfa_hyperparams['weights']['mu'].data.numpy()
confidence = 0.95

In [17]:
### task classification using leave one out runs on NTFA###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
summary_subject_chance_means = []
for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        if i not in validation_blocks:
            block = dtfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(all_weights_dtfa[i,:,:].mean(0))
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                 session_labels = session_labels,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())
    summary_subject_chance_means.append(np.array(overall_result_chance).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("NTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("NTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.4f +- %1.4f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("NTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

print("Overall Dummy Mean/CI Score: %1.4f +- %1.4f" % (
np.array(summary_result_chance).mean(),
stats.sem(summary_subject_chance_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_chance_means) - 1)))

ntfa_summary_result = summary_result
dummy_summary_result = summary_result_chance
ntfa_summary_means = summary_subject_means
dummy_summary_means = summary_subject_chance_means

Processing Subject 10
Processing Heights
Scores: 0.9896 +- 0.0147
Chance Scores: 0.5000 +- 0.1531
Processing Social threat
Scores: 0.9479 +- 0.0531
Chance Scores: 0.5833 +- 0.1284
Processing Spiders
Scores: 1.0000 +- 0.0000
Chance Scores: 0.3958 +- 0.1284
...
Overall Score: 0.9792 +- 0.0390
Overall Dummy Score: 0.4931 +- 0.1571


Processing Subject 11
Processing Heights
Scores: 0.8512 +- 0.1388
Chance Scores: 0.5149 +- 0.0800
Processing Social threat
Scores: 0.9003 +- 0.0331
Chance Scores: 0.4494 +- 0.0374
Processing Spiders
Scores: 0.8576 +- 0.0762
Chance Scores: 0.3889 +- 0.0804
...
Overall Score: 0.8697 +- 0.0959
Overall Dummy Score: 0.4511 +- 0.0860


Processing Subject 12
Processing Heights
Scores: 0.4896 +- 0.0820
Chance Scores: 0.5347 +- 0.0491
Processing Social threat
Scores: 0.5327 +- 0.1021
Chance Scores: 0.3929 +- 0.1051
Processing Spiders
Scores: 0.6146 +- 0.1699
Chance Scores: 0.6220 +- 0.1568
...
Overall Score: 0.5456 +- 0.1343
Overall Dummy Score: 0.5165 +- 0.1470


Proc

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8368 +- 0.0682
Chance Scores: 0.5208 +- 0.0780
Processing Social threat
Scores: 0.9345 +- 0.0512
Chance Scores: 0.4911 +- 0.1397
Processing Spiders
Scores: 0.9479 +- 0.0737
Chance Scores: 0.5357 +- 0.1089
...
Overall Score: 0.9064 +- 0.0818
Overall Dummy Score: 0.5159 +- 0.1132


Processing Subject 16
Processing Heights
Scores: 0.9653 +- 0.0260
Chance Scores: 0.4722 +- 0.0708
Processing Social threat
Scores: 0.9896 +- 0.0147
Chance Scores: 0.6369 +- 0.1540
Processing Spiders
Scores: 0.9583 +- 0.0295
Chance Scores: 0.4375 +- 0.0884
...
Overall Score: 0.9711 +- 0.0277
Overall Dummy Score: 0.5155 +- 0.1405


Processing Subject 17
Processing Heights
Scores: 0.9062 +- 0.0442
Chance Scores: 0.5694 +- 0.0597
Processing Social threat
Scores: 0.8259 +- 0.0384
Chance Scores: 0.5982 +- 0.0768
Processing Spiders
Scores: 0.8780 +- 0.0400
Chance Scores: 0.4940 +- 0.0800
...
Overall Score: 0.8700 +- 0.0528
Overall Dummy Score: 0.5539 +- 0.0850


Processing Subject 18
Processing Heights
Scor

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8147 +- 0.0290
Chance Scores: 0.4286 +- 0.1786
Processing Social threat
Scores: 0.8147 +- 0.0290
Chance Scores: 0.3661 +- 0.0089
Processing Spiders
Scores: 0.8490 +- 0.0677
Chance Scores: 0.3333 +- 0.0833
...
Overall Score: 0.8261 +- 0.0485
Overall Dummy Score: 0.3760 +- 0.1205


Processing Subject 25
Processing Heights
Scores: 0.7604 +- 0.0390
Chance Scores: 0.5417 +- 0.1179
Processing Social threat
Scores: 0.9340 +- 0.0723
Chance Scores: 0.2917 +- 0.0780
Processing Spiders
Scores: 0.9896 +- 0.0147
Chance Scores: 0.4524 +- 0.1207
...
Overall Score: 0.8947 +- 0.1089
Overall Dummy Score: 0.4286 +- 0.1490


Processing Subject 26
Processing Heights
Scores: 0.6406 +- 0.1094
Chance Scores: 0.4554 +- 0.0446
Processing Social threat
Scores: 0.7604 +- 0.0521
Chance Scores: 0.5000 +- 0.0625
Processing Spiders
Scores: 0.8058 +- 0.0871
Chance Scores: 0.4598 +- 0.1027
...
Overall Score: 0.7356 +- 0.1108
Overall Dummy Score: 0.4717 +- 0.0767


Processing Subject 28
Processing Heights
Scor

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8497 +- 0.0331
Chance Scores: 0.4345 +- 0.0984
Processing Social threat
Scores: 0.9062 +- 0.1112
Chance Scores: 0.4286 +- 0.0526
Processing Spiders
Scores: 0.9653 +- 0.0260
Chance Scores: 0.4792 +- 0.1284
...
Overall Score: 0.9071 +- 0.0833
Overall Dummy Score: 0.4474 +- 0.1008


Processing Subject 6
Processing Heights
Scores: 0.4509 +- 0.1065
Chance Scores: 0.4524 +- 0.0552
Processing Social threat
Scores: 0.2679 +- 0.0631
Chance Scores: 0.5119 +- 0.2163
Processing Spiders
Scores: 0.6111 +- 0.0856
Chance Scores: 0.5625 +- 0.1350
...
Overall Score: 0.4433 +- 0.1650
Overall Dummy Score: 0.5089 +- 0.1572


Processing Subject 7
Processing Heights
Scores: 0.4375 +- 0.0000
Chance Scores: 0.5312 +- 0.0938
Processing Social threat
Scores: 0.4375 +- 0.0938
Chance Scores: 0.5000 +- 0.0625
Processing Spiders
Scores: 0.3594 +- 0.0156
Chance Scores: 0.5938 +- 0.0938
...
Overall Score: 0.4115 +- 0.0661
Overall Dummy Score: 0.5417 +- 0.0932


Processing Subject 8
Processing Heights
Scores:

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

In [18]:
### task classification using leave one out runs on input data###
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []

for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        block = dtfa._blocks[i]
        if i not in validation_blocks:
            if block.subject == s:
                session_labels.append(block.run)
                data.append(dtfa.voxel_activations[i].data.numpy().mean(0))
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                    session_labels = session_labels,
                                                                    feature_select=True,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("Voxel Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("Voxel Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.2f +- %1.2f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("Voxel Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

voxel_summary_result = summary_result
voxel_summary_means = summary_subject_means

Processing Subject 10
Processing Heights
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7930 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7162 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.9583 +- 0.0589
Chance Scores: 0.5417 +- 0.0780
Processing Social threat
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7162 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 5944  7106  7134 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.4018 +- 0.0063
Chance Scores: 0.4107 +- 0.0718
Processing Spiders
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 5944  7106  7134 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 5944  7106  7134 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.6042 +- 0.1699
Chance Scores: 0.5347 +- 0.1133
...
Overall Score: 0.4965 +- 0.1605
Overall Dummy Score: 0.4868 +- 0.1361


...
Voxel Summary Score Across Subjects, For Category Heights : 0.8001 +- 0.2265
Voxel Summary Score Across Subjects, For Category Social threat : 0.8217 +- 0.2367
Voxel Summary Score Across Subjects, For Category Spiders : 0.8639 +- 0.1865
...
Voxel Summary Score Across Subjects, Across Tasks: 0.8285 +- 0.2193
Overall Dummy Score: 0.50 +- 0.16
...
Voxel Summary Mean/CI, Across Tasks: 0.8201 +- 0.0927


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [19]:
### task classification using leave one out runs on input data with KBest###
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
feature_selection = SelectKBest(f_classif, k=500)

for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        block = dtfa._blocks[i]
        if i not in validation_blocks:
            if block.subject == s:
                session_labels.append(block.run)
                data.append(dtfa.voxel_activations[i].data.numpy().mean(0))
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                    session_labels = session_labels,
                                                                    feature_select=True,
                                                                    feature_selection=feature_selection,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("Voxel Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("Voxel Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.2f +- %1.2f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("Voxel Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

voxel_summary_result = summary_result
voxel_summary_means = summary_subject_means

Processing Subject 10
Processing Heights
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7930 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7162 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.9688 +- 0.0442
Chance Scores: 0.5833 +- 0.1640
Processing Social threat
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7930 ... 81635 81636 81637] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 7106  7134  7162 ... 81635 81636 81637] are constant.
  UserWarning)


TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 5944  7106  7134 ... 81635 81636 81637] are constant.
  UserWarning)


TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 5944  7106  7134 ... 81635 81636 81637] are constant.
  UserWarning)


TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.5729 +- 0.1538
Chance Scores: 0.4514 +- 0.0687
...
Overall Score: 0.5298 +- 0.1466
Overall Dummy Score: 0.5086 +- 0.1293


...
Voxel Summary Score Across Subjects, For Category Heights : 0.8018 +- 0.2062
Voxel Summary Score Across Subjects, For Category Social threat : 0.8172 +- 0.2202
Voxel Summary Score Across Subjects, For Category Spiders : 0.8538 +- 0.1833
...
Voxel Summary Score Across Subjects, Across Tasks: 0.8243 +- 0.2050
Overall Dummy Score: 0.49 +- 0.14
...
Voxel Summary Mean/CI, Across Tasks: 0.8163 +- 0.0841


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [20]:
stats.ttest_ind(ntfa_summary_means,dummy_summary_means)

Ttest_indResult(statistic=7.088679720389954, pvalue=1.4118143630149038e-08)

In [21]:
from scipy.stats import pearsonr
from scipy.special import betainc
def corrcoef(matrix):
    r = np.corrcoef(matrix)
    rf = r[np.triu_indices(r.shape[0], 1)]
    df = matrix.shape[1] - 2
    ts = rf * rf * (df / (1 - rf * rf))
    pf = betainc(0.5 * df, 0.5, df / (df + ts))
    p = np.zeros(shape=r.shape)
    p[np.triu_indices(p.shape[0], 1)] = pf
    p[np.tril_indices(p.shape[0], -1)] = p.T[np.tril_indices(p.shape[0], -1)]
    p[np.diag_indices(p.shape[0])] = np.ones(p.shape[0])
    return r, p


In [22]:
### functional connectivity using NTFA ###

all_factor_centers = dtfa_hyperparams['factor_centers']['mu'].data.numpy()
all_factor_widths = np.exp(dtfa_hyperparams['factor_log_widths']['mu'].data.numpy())
spider_matrix = []
social_matrix = []
heights_matrix = []
all_matrix = []

spider_matrix_p = []
social_matrix_p = []
heights_matrix_p = []
all_matrix_p = []
for i in range(dtfa.num_blocks):
    r,p = corrcoef(all_weights_dtfa[i,:,:].T)
    p = p < 0.05
    all_matrix.append(r)
    all_matrix_p.append(p)
    if block_task_labeler(dtfa._blocks[i]) == 'Spiders':
        r,p = corrcoef(all_weights_dtfa[i,:,:].T)
        p = p < 0.05
        spider_matrix.append(r)
        spider_matrix_p.append(p)
    elif  block_task_labeler(dtfa._blocks[i]) == 'Social threat':
        r,p = corrcoef(all_weights_dtfa[i,:,:].T)
        p = p < 0.05
        social_matrix.append(r)
        social_matrix_p.append(p)
    elif  block_task_labeler(dtfa._blocks[i]) == 'Heights':
        r,p = corrcoef(all_weights_dtfa[i,:,:].T)
        p = p < 0.05
        heights_matrix.append(r)
        heights_matrix_p.append(p)

        
        
spider_matrix = (np.array(spider_matrix))
social_matrix = (np.array(social_matrix))
heights_matrix = (np.array(heights_matrix))
all_matrix = np.array(all_matrix)

spider_matrix_p = (np.array(spider_matrix_p))
social_matrix_p = (np.array(social_matrix_p))
heights_matrix_p = (np.array(heights_matrix_p))
all_matrix_p = np.array(all_matrix_p)

spider_matrix = spider_matrix * spider_matrix_p
social_matrix = social_matrix * social_matrix_p
heights_matrix = heights_matrix * heights_matrix_p
all_matrix = all_matrix * all_matrix_p

all_vectors = np.zeros(shape=(all_matrix.shape[0],4950))
all_vectors_p = np.zeros(shape=(all_matrix_p.shape[0],4950))

for i in range(all_matrix.shape[0]):
    all_vectors[i,:] = distance.squareform(all_matrix[i,:,:],checks=False)
    all_vectors_p[i,:] = distance.squareform(all_matrix_p[i,:,:],checks=False)

spider_vectors = np.zeros(shape=(spider_matrix.shape[0],4950))
spider_vectors_p = np.zeros(shape=(spider_matrix_p.shape[0],4950))

for i in range(spider_matrix.shape[0]):
    spider_vectors[i,:] = distance.squareform(spider_matrix[i,:,:],checks=False)
    spider_vectors_p[i,:] = distance.squareform(spider_matrix_p[i,:,:],checks=False)


social_vectors = np.zeros(shape=(social_matrix.shape[0],4950))
social_vectors_p = np.zeros(shape=(social_matrix_p.shape[0],4950))

for i in range(social_matrix.shape[0]):
    social_vectors[i,:] = distance.squareform(social_matrix[i,:,:],checks=False)
    social_vectors_p[i,:] = distance.squareform(social_matrix_p[i,:,:],checks=False)

    
heights_vectors = np.zeros(shape=(heights_matrix.shape[0],4950))
heights_vectors_p = np.zeros(shape=(heights_matrix_p.shape[0],4950))

for i in range(heights_matrix.shape[0]):
    heights_vectors[i,:] = distance.squareform(heights_matrix[i,:,:],checks=False)
    heights_vectors_p[i,:] = distance.squareform(heights_matrix_p[i,:,:],checks=False)
    
spider_vectors_mean = spider_vectors.mean(0)
spider_vectors_std = spider_vectors.std(0)
social_vectors_mean = social_vectors.mean(0)
social_vectors_std = social_vectors.std(0)
heights_vectors_mean = heights_vectors.mean(0)
heights_vectors_std = heights_vectors.std(0)

In [23]:
all_vectors[0].shape

(4950,)

In [24]:
### task classification using leave one out runs on NTFA using functional connectivity###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
summary_subject_chance_means = []
for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        if i not in validation_blocks:
            block = dtfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(all_vectors[i])
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                    session_labels = session_labels,
                                                                   scoring="roc_auc",
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())
    summary_subject_chance_means.append(np.array(overall_result_chance).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("NTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("NTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.4f +- %1.4f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("NTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

print("Overall Dummy Mean/CI Score: %1.4f +- %1.4f" % (
np.array(summary_subject_chance_means).mean(),
stats.sem(summary_subject_chance_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_chance_means) - 1)))

ntfa_summary_result = summary_result
dummy_summary_result = summary_result_chance
ntfa_summary_means = summary_subject_means
dummy_summary_means = summary_subject_chance_means

Processing Subject 10
Processing Heights
Scores: 0.8125 +- 0.0884
Chance Scores: 0.5000 +- 0.0510
Processing Social threat
Scores: 0.9271 +- 0.0531
Chance Scores: 0.5000 +- 0.0884
Processing Spiders


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.8854 +- 0.0737
Chance Scores: 0.6250 +- 0.0884
...
Overall Score: 0.8750 +- 0.0872
Overall Dummy Score: 0.5417 +- 0.0977


Processing Subject 11
Processing Heights
Scores: 0.5923 +- 0.0900
Chance Scores: 0.4524 +- 0.0845
Processing Social threat
Scores: 0.8393 +- 0.0636
Chance Scores: 0.6399 +- 0.0345
Processing Spiders


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.6910 +- 0.0723
Chance Scores: 0.4931 +- 0.0597
...
Overall Score: 0.7075 +- 0.1269
Overall Dummy Score: 0.5284 +- 0.1022


Processing Subject 12
Processing Heights
Scores: 0.3368 +- 0.0344
Chance Scores: 0.5208 +- 0.0780
Processing Social threat
Scores: 0.3229 +- 0.1495
Chance Scores: 0.5774 +- 0.1223
Processing Spiders


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.9306 +- 0.0273
Chance Scores: 0.4236 +- 0.1874
Processing Spiders
Scores: 0.8051 +- 0.0172
Chance Scores: 0.4286 +- 0.0526
...
Overall Score: 0.8647 +- 0.0590
Overall Dummy Score: 0.4626 +- 0.1552


Processing Subject 9
Processing Heights
Scores: 0.3676 +- 0.0503
Chance Scores: 0.4732 +- 0.0263
Processing Social threat


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sk

Scores: 0.2723 +- 0.0159
Chance Scores: 0.6012 +- 0.0611
Processing Spiders
Scores: 0.5764 +- 0.0687
Chance Scores: 0.4375 +- 0.2339
...
Overall Score: 0.4054 +- 0.1365
Overall Dummy Score: 0.5040 +- 0.1570


...
NTFA Summary Score Across Subjects, For Category Heights : 0.6853 +- 0.2205
NTFA Summary Score Across Subjects, For Category Social threat : 0.7275 +- 0.2361
NTFA Summary Score Across Subjects, For Category Spiders : 0.7417 +- 0.1878
...
NTFA Summary Score Across Subjects, Across Tasks: 0.7181 +- 0.2171
Overall Dummy Score: 0.5087 +- 0.1298
...
NTFA Summary Mean/CI, Across Tasks: 0.7148 +- 0.0751
Overall Dummy Mean/CI Score: 0.5067 +- 0.0258


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [25]:
### task classification using leave one out runs on NTFA using functional connectivity with feature selection###
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
summary_subject_chance_means = []
feature_selection = SelectKBest(f_classif, k=500)
for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        if i not in validation_blocks:
            block = dtfa._blocks[i]
            if block.subject == s:
                session_labels.append(block.run)
                data.append(np.arctanh(all_vectors[i]))
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                    session_labels = session_labels,
                                                                   scoring="roc_auc",feature_select=True,
                                                                   feature_selection=feature_selection,
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())
    summary_subject_chance_means.append(np.array(overall_result_chance).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        category_result[c].extend(s[c])
    print("NTFA Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("NTFA Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.4f +- %1.4f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("NTFA Summary Mean/CI, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

print("Overall Dummy Mean/CI Score: %1.4f +- %1.4f" % (
np.array(summary_subject_chance_means).mean(),
stats.sem(summary_subject_chance_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_chance_means) - 1)))

ntfa_summary_result = summary_result
dummy_summary_result = summary_result_chance
ntfa_summary_means = summary_subject_means
dummy_summary_means = summary_subject_chance_means

Processing Subject 10
Processing Heights
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  10   83   86  108  185  247  281  300  301  318  327  343  381  420
  422  453  472  597  600  608  636  663  670  675  699  712  717  732
  768  771  846  848  856  883  897  900  913  922  927  938  947  952
  953  956  959  975  978  990  993  994  999 1004 1008 1019 1021 1022
 1026 1031 1042 1054 1059 1063 1070 1084 1091 1097 1104 1105 1109 1111
 1113 1119 1120 1173 1208 1218 1231 1246 1253 1260 1280 1283 1287 1368
 1389 1401 1451 1462 1469 1470 1473 1484 1500 1505 1509 1516 1517 1524
 1526 1532 1535 1546 1557 1561 1613 1628 1638 1719 1728 1770 1861 1872
 1900 1949 1954 1975 1992 2112 2159 2162 2166 2181 2189 2192 2205 2207
 2211 2212 2213 2216 2220 2230 2231 2257 2394 2395 2397 2401 2411 2425
 2427 2429 2430 2432 2436 2447 2450 2452 2454 2455 2458 2462 2463 2472
 2488 2536 2557 2559 2591 2595 2609 2672 2675 2677 2678 2681 2684 2697


Scores: 0.7812 +- 0.0442
Chance Scores: 0.6042 +- 0.0780
Processing Social threat
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.9167 +- 0.0642
Chance Scores: 0.4375 +- 0.0884
Processing Spiders
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.8542 +- 0.0780
Chance Scores: 0.4583 +- 0.1062
...
Overall Score: 0.8507 +- 0.0843

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  52  119  205  247  262  301  318  327  343  378  420  438  472  504
  514  567  570  578  579  581  588  589  597  610  663  670  674  675
  692  699  717  749  752  766  768  780  790  792  819  848  879  883
  897  902  905  922  924  929  930  936  938  945  947  948  949  951
  953  954  958  960  968  969  983  988  992  994  995  996  997  999
 1004 1010 1013 1014 1016 1019 1022 1024 1028 1029 1050 1051 1054 1056
 1067 1070 1072 1095 1096 1108 1120 1162 1218 1227 1232 1233 1237 1247
 1248 1254 1274 1279 1280 1287 1290 1300 1368 1389 1475 1500 1509 1515
 1517 1520 1526 1535 1546 1554 1561 1582 1617 1628 1635 1638 1660 1663
 1688 1712 1725 1752 1755 1760 1769 1773 1789 1794 1837 1853 1857 1868
 1884 1940 1992 2015 2068 2103 2107 2143 2149 2159 2180 2181 2182 2191
 2193 2197 2205 2207 2217 2218 2219 2227 2230 2231 2233 2247 2257 2306


TRAIN: [ 1  2  4  5  7  9 10 12 13 15 16 18 19 21 22 24 25 26 28 30 31 33 34] TEST: [ 0  3  6  8 11 14 17 20 23 27 29 32]
TRAIN: [ 0  1  3  4  6  8 10 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33] TEST: [ 2  5  7  9 13 16 19 22 25 28 31 34]
Scores: 0.5699 +- 0.0779
Chance Scores: 0.3423 +- 0.0940
Processing Social threat
TRAIN: [ 0  2  3  5  6  7  8  9 11 13 14 16 17 19 20 22 23 25 27 28 29 31 32 34] TEST: [ 1  4 10 12 15 18 21 24 26 30 33]
TRAIN: [ 1  2  4  5  7  9 10 12 13 15 16 18 19 21 22 24 25 26 28 30 31 33 34] TEST: [ 0  3  6  8 11 14 17 20 23 27 29 32]
TRAIN: [ 0  1  3  4  6  8 10 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33] TEST: [ 2  5  7  9 13 16 19 22 25 28 31 34]
Scores: 0.7976 +- 0.0673
Chance Scores: 0.6190 +- 0.0800
Processing Spiders
TRAIN: [ 0  2  3  5  6  7  8  9 11 13 14 16 17 19 20 22 23 25 27 28 29 31 32 34] TEST: [ 1  4 10 12 15 18 21 24 26 30 33]


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   2    6   10   27   31   34   56   67   84   92  190  195  204  216
  222  272  279  290  318  326  354  375  395  420  478  491  501  532
  543  566  585  595  609  620  638  675  676  692  721  734  854  867
  868  883  898  915  937  947  948  949  950  951  952  961  963  964
  965  974  977  978  980  982  984  995  996  997  998 1001 1002 1003
 1004 1007 1009 1012 1013 1024 1029 1030 1050 1053 1057 1061 1067 1069
 1071 1075 1080 1095 1099 1108 1122 1128 1135 1136 1138 1139 1140 1153
 1160 1171 1173 1175 1191 1223 1243 1250 1277 1279 1282 1293 1373 1430
 1593 1595 1716 1734 1738 1797 1817 1860 1862 1875 1917 1932 1948 1952
 1954 1959 1961 1965 1967 1969 1972 1979 1984 1987 1989 2001 2028 2100
 2103 2162 2188 2192 2217 2234 2306 2322 2364 2381 2394 2402 2404 2406
 2409 2418 2422 2423 2426 2431 2433 2442 2443 2447 2448 2454 2455 2472


TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.3348 +- 0.0726
Chance Scores: 0.3512 +- 0.2315
Processing Social threat
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  22  107  110  134  185  196  204  227  278  285  383  420  422  476
  484  504  516  567  572  588  709  717  732  748  780  855  862  882
  893  898  932  936  945  946  951  952  953  959  960  961  964  965
  975  979  986  991  992  995  997  998  999 1003 1007 1015 1019 1022
 1025 1028 1030 1031 1038 1041 1070 1071 1086 1104 1106 1115 1218 1227
 1232 1233 1254 1259 1261 1274 1278 1294 1317 1368 1389 1502 1512 1515
 1517 1521 1535 1648 1728 1774 1794 1862 1868 1952 2008 2035 2062 2091
 2093 2099 2107 2110 2174 2181 2184 2186 2204 2211 2213 2215 2231 2234
 2235 2240 2242 2260 2326 2331 2397 2401 2404 2427 2446 2447 2452 2503
 2526 2541 2607 2663 2674 2712 2715 2722 2728 2737 2740 2762 2876 2883
 2884 2885 2887 2890 2892 2897 2898 2903 2906 2911 2915 2919 2925 2928
 2929 2930 2931 2961 2981 3000 3010 3029 3030 3045 3052 3059 3068 3070


Scores: 0.2619 +- 0.0537
Chance Scores: 0.5595 +- 0.0842
Processing Spiders
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]
TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.5035 +- 0.1459
Chance Scores: 0.4931 +- 0.0597
...
Overall Score: 0.3667 +- 0.1416
Overall Dummy Score: 0.4679 +- 0.1702


...
NTFA Summary Score Across Subjects, For Category Heights : 0.6884 +- 0.2169
NTFA Summary Score Across Subjects, For Category Social threat : 0.7281 +- 0.2255
NTFA Summary Score Across Subjects, For Category Spiders : 0.7193 +- 0.2064
...
NTFA Summary Score Across Subjects, Across Tasks: 0.7119 +- 0.2171
Overall Dummy Score: 0.4791 +- 0.1542
...
NTFA Summary Mean/CI, Across Tasks: 0.7093 +- 0.0765
Overall Dummy Mean/CI

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [  18   22   27   83  129  207  227  262  278  413  422  449  469  489
  492  504  514  516  572  588  597  717  738  839  855  856  858  868
  882  897  898  946  953  954  956  959  964  967  975  979  982  991
  993  996  997  999 1007 1015 1019 1028 1070 1113 1115 1144 1158 1173
 1210 1216 1218 1222 1245 1250 1251 1254 1274 1287 1294 1295 1315 1317
 1354 1389 1467 1470 1476 1482 1502 1517 1530 1674 1742 2035 2177 2178
 2192 2210 2211 2213 2220 2221 2240 2245 2250 2260 2326 2329 2331 2378
 2397 2403 2405 2425 2427 2439 2446 2452 2456 2458 2459 2464 2467 2523
 2526 2534 2536 2541 2604 2607 2674 2676 2684 2688 2712 2714 2722 2723
 2728 2775 2798 2804 2838 2854 2873 2879 2883 2887 2891 2898 2899 2900
 2904 2909 2913 2925 2928 2930 2931 2950 2961 2981 2997 3000 3002 3008
 3009 3010 3029 3030 3045 3052 3065 3068 3069 3129 3178 3207 3223 3238


In [26]:
parcellation_file = '/data/zulqarnain/parcellation_masks/affvids_parcellation_180.nii.gz'

In [27]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

all_matrix = []
all_matrix_p = []
for i in range(dtfa.num_blocks):
#     print (i)
    activations = dtfa.voxel_activations[i].data.numpy()
    locations = dtfa.voxel_locations.data.numpy()
    block_img = utils.cmu2nii(activations,locations,dtfa._templates[i])
    masker = NiftiLabelsMasker(labels_img=parcellation_file)
    time_series = masker.fit_transform(block_img)
    r,p = corrcoef(time_series.T)
    p = p < 0.05
#     all_matrix.append(r)
    if np.isnan(r).any():
        print (i)
        r = np.nan_to_num(r)
        p = np.nan_to_num(p)
        all_matrix_p.append(p)
        all_matrix.append(r)
all_matrix = np.array(all_matrix)
all_matrix_p = np.array(all_matrix_p)
all_matrix = all_matrix  * all_matrix_p
all_vectors = np.zeros(shape=(all_matrix.shape[0],16110))
all_vectors_p = np.zeros(shape=(all_matrix_p.shape[0],16110))

for i in range(all_matrix.shape[0]):
    all_vectors[i,:] = distance.squareform(all_matrix[i,:,:],checks=False)
    all_vectors_p[i,:] = distance.squareform(all_matrix_p[i,:,:],checks=False)

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [28]:
### task classification using leave one out runs on input data###
confidence = 0.95
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []
feature_selection = SelectKBest(f_classif, k=500)

for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        block = dtfa._blocks[i]
        if i not in validation_blocks:
            if block.subject == s:
                session_labels.append(block.run)
                data.append(np.arctanh(all_vectors[i]))
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                   session_labels=session_labels,
                                                                   cv = cv,
                                                                   scoring="roc_auc",
                                                                   feature_select = True,
                                                                   feature_selection=feature_selection,
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        if c in s.keys():
            category_result[c].extend(s[c])
    print("Voxel Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("Voxel Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.2f +- %1.2f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("Voxel Summary Mean/SE, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

voxel_summary_result = summary_result
voxel_summary_means = summary_subject_means

Processing Subject 10
Processing Heights
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   38    73   100 ... 16081 16084 16096] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   72   120   122 ... 16088 16093 16104] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]
Scores: 0.7396 +- 0.0780
Chance Scores: 0.4375 +- 0.1350
Processing Social threat
TRAIN: [ 0  1  3  4  6  7  9 11 12 14 15 17 18 20 21 23 24 26 27 29 30 32 33 34] TEST: [ 2  5  8 10 13 16 19 22 25 28 31 35]
TRAIN: [ 1  2  3  5  6  8  9 10 12 13 15 16 18 19 21 22 24 25 27 28 30 31 33 35] TEST: [ 0  4  7 11 14 17 20 23 26 29 32 34]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   38    73   100 ... 16081 16084 16096] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   72   120   122 ... 16088 16093 16104] are constant.
  UserWarning)


TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 22 23 25 26 28 29 31 32 34 35] TEST: [ 1  3  6  9 12 15 18 21 24 27 30 33]


...

TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.3497 +- 0.1188
Chance Scores: 0.5387 +- 0.0276
Processing Social threat
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   90    91   120 ... 16091 16092 16096] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   91   120   164 ... 16080 16081 16086] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.3661 +- 0.1575
Chance Scores: 0.4970 +- 0.1810
Processing Spiders
TRAIN: [ 1  2  3  4  6  8  9 10 12 13 15 16 18 20 22 23 25 26 28 29 31 32 33 34] TEST: [ 0  5  7 11 14 17 19 21 24 27 30]
TRAIN: [ 0  2  4  5  7  8 10 11 13 14 16 17 19 20 21 23 24 26 27 29 30 32 34] TEST: [ 1  3  6  9 12 15 18 22 25 28 31 33]


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   90    91   120 ... 16091 16092 16096] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [   91   120   164 ... 16080 16081 16086] are constant.
  UserWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/feature_selection/_u

TRAIN: [ 0  1  3  5  6  7  9 11 12 14 15 17 18 19 21 22 24 25 27 28 30 31 33] TEST: [ 2  4  8 10 13 16 20 23 26 29 32 34]
Scores: 0.3750 +- 0.1169
Chance Scores: 0.5139 +- 0.1678
...
Overall Score: 0.3636 +- 0.1328
Overall Dummy Score: 0.5165 +- 0.1444


...
Voxel Summary Score Across Subjects, For Category Heights : 0.5697 +- 0.1798
Voxel Summary Score Across Subjects, For Category Social threat : 0.6478 +- 0.2365
Voxel Summary Score Across Subjects, For Category Spiders : 0.6656 +- 0.2195
...
Voxel Summary Score Across Subjects, Across Tasks: 0.6277 +- 0.2173
Overall Dummy Score: 0.51 +- 0.15
...
Voxel Summary Mean/SE, Across Tasks: 0.6194 +- 0.0721


In [29]:
### task classification using leave one out runs on input data no feature selection###
confidence = 0.95
cv = LeaveOneGroupOut()
summary_result = []
summary_result_chance = []
summary_result_category = []
summary_subject_means = []

for s in dtfa.subjects():
    print("Processing Subject %d" % (s))
    session_labels = []
    data = []
    targets = []
    for i in range(dtfa.num_blocks):
        block = dtfa._blocks[i]
        if i not in validation_blocks:
            if block.subject == s:
                session_labels.append(block.run)
                data.append(all_vectors[i])
                targets.append(task_labeler(block.task))
    data = np.array(data)
    overall_result,overall_result_chance,category_result = classification_pipeline(data=data,
                                                                   targets=targets,
                                                                   session_labels=session_labels,
                                                                   cv = cv,
                                                                   scoring="roc_auc",
                                                                   feature_select = False,
                                                                   print_all=True)
    summary_result_category.append(category_result)
    summary_result.extend(overall_result)
    summary_result_chance.extend(overall_result_chance)
    summary_subject_means.append(np.array(overall_result).mean())

print ('...')
category_result = {key:[] for key in np.unique(targets)}
for c in np.unique(targets):
    for s in summary_result_category:
        if c in s.keys():
            category_result[c].extend(s[c])
    print("Voxel Summary Score Across Subjects, For Category %s : %1.4f +- %1.4f" % (c,
np.array(category_result[c]).mean(),
np.array(category_result[c]).std()))
    
print ('...')
print("Voxel Summary Score Across Subjects, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_result).mean(),
np.array(summary_result).std()))

print("Overall Dummy Score: %1.2f +- %1.2f" % (
np.array(summary_result_chance).mean(),
np.array(summary_result_chance).std()))

print ('...')
print("Voxel Summary Mean/SE, Across Tasks: %1.4f +- %1.4f" % (
np.array(summary_subject_means).mean(),
stats.sem(summary_subject_means) 
    * stats.t.ppf((1 + confidence) / 2, len(summary_subject_means) - 1)))

voxel_summary_result = summary_result
voxel_summary_means = summary_subject_means

Processing Subject 10
Processing Heights


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Scores: 0.8021 +- 0.0780
Chance Scores: 0.5000 +- 0.1350
Processing Social threat


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Scores: 0.7917 +- 0.1031
Chance Scores: 0.4375 +- 0.1021
Processing Spiders


...

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Scores: 0.3810 +- 0.0770
Chance Scores: 0.5982 +- 0.0981
Processing Social threat


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


Scores: 0.2917 +- 0.1559
Chance Scores: 0.4524 +- 0.1545
Processing Spiders
Scores: 0.3681 +- 0.1254
Chance Scores: 0.4722 +- 0.1277
...
Overall Score: 0.3469 +- 0.1299
Overall Dummy Score: 0.5076 +- 0.1441


...
Voxel Summary Score Across Subjects, For Category Heights : 0.5917 +- 0.1862
Voxel Summary Score Across Subjects, For Category Social threat : 0.6419 +- 0.2400
Voxel Summary Score Across Subjects, For Category Spiders : 0.6699 +- 0.2404
...
Voxel Summary Score Across Subjects, Across Tasks: 0.6345 +- 0.2260
Overall Dummy Score: 0.52 +- 0.16
...
Voxel Summary Mean/SE, Across Tasks: 0.6271 +- 0.0758


/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.7/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
